以下のデータを取得して、`data`ディレクトリに配置。

---

__120 years of Olympic history: athletes and results__  
  URL:  
    ・https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results  
  DATA:  
    ・athlete_events.csv

---

__World cities database__  
  URL:  
    ・https://www.kaggle.com/juanmah/world-cities  
  DATA:  
    ・worldcities.csv

In [ ]:
%matplotlib inline

import folium
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

### 1. データの概要を掴む
とにかく大雑把にデータの特徴を把握しよう。

オリンピック選手のデータをロード。

In [ ]:
athlete_events_df = pd.read_csv('data/athlete_events.csv')

len(athlete_events_df)

データの中身を確認。

In [ ]:
athlete_events_df.head()

数値属性の集計情報を確認。  
何か特徴が読み取れるか？

In [ ]:
athlete_events_df.describe()

ヒストグラムで表示。  
何か特徴が読み取れるか？

データを加工するたびにグラフにして全体象を把握する。

In [ ]:
athlete_events_df.hist(bins=50, figsize=(20, 12))

### 2. 地図を使ってデータを把握する
地理情報がある場合は、地図上にデータを表示させてみる。

開催都市を地図上に表示。
- 夏は赤系、冬は青系の円を配置。
- 参加選手数に合わせて円を大きくする。
- 複数回開催している場合は色分けする。

In [ ]:
# 都市、ゲームでグループ分け。
city_games_df = athlete_events_df.groupby(['City', 'Year', 'Games']).agg(number_of_athletes=('ID', 'count'))
city_games_df.reset_index(inplace=True)

# 色分け用にseason列（夏季か冬季か？）を追加。
city_games_df['season'] = city_games_df['Games'].str.split().str[1]

city_games_df.head(10)

複数回開催している都市があるので、何回目の開催であるかを示す列を追加。

In [ ]:
city_games_df['number_of_times'] = city_games_df.groupby(['City', 'season']).cumcount() + 1

city_games_df.head(10)

世界の都市の位置情報データをロード。

In [ ]:
worldcities_df = pd.read_csv('data/worldcities.csv')

len(worldcities_df)

データを確認。

In [ ]:
worldcities_df.head(10)

`city_ascii`が英語名のようなので、`city_ascii`を都市名として利用。  
同名の都市がないかチェック。

In [ ]:
worldcities_df.duplicated(subset='city_ascii').sum()

都市名の重複が相当ある模様。  
どんな重複があるのか確認。

In [ ]:
worldcities_df[worldcities_df.duplicated(subset='city_ascii')].sort_values('city_ascii')

とりあえず機械的に、都市名が重複している場合は人口数の多いものだけ残すようにする。

In [ ]:
# 都市名、人口の昇順にする。
worldcities_df.sort_values(['city_ascii', 'population'], inplace=True)

# 重複都市名の最後の行、つまり一番人口が多いものを残す。
worldcities_df.drop_duplicates(subset='city_ascii', keep='last', inplace=True)

もう一度重複数を確認。

In [ ]:
worldcities_df.duplicated(subset='city_ascii').sum()

必要な列のみ残す。

In [ ]:
worldcities_df = worldcities_df[['city_ascii', 'lat', 'lng']]

都市名をキーとしてデータフレームを結合。

In [ ]:
city_game_coords_df = pd.merge(city_games_df, worldcities_df, left_on=['City'], right_on=['city_ascii'], how='left')

city_game_coords_df.head(10)

緯度経度が入っていないものをチェック。

In [ ]:
city_game_coords_df[city_game_coords_df['lat'].isna()]

数が少ないので、個別対応する。
- Antwerpen → Antwerp
- Athina → Athens 
- Chamonix → Chamonix-Mont-Blanc
- Garmisch-Partenkirchen → Garmisch-Partenkirchen (47.492, 11.0931)
- Moskva → Moscow
- Sankt Moritz → St. Moritz (46.578, 9.8353)
- Squaw Valley → Palisades Tahoe (39.208, -120.2132)
- Torino → Turin

In [ ]:
city_game_coords_df.loc[(city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Antwerpen"), ['city_ascii', 'lat', 'lng']] = \
    worldcities_df.query('city_ascii == "Antwerp"').values

city_game_coords_df.loc[(city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Athina"), ['city_ascii', 'lat', 'lng']] = \
    worldcities_df.query('city_ascii == "Athens"').values

city_game_coords_df.loc[(city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Chamonix"), ['city_ascii', 'lat', 'lng']] = \
    worldcities_df.query('city_ascii == "Chamonix-Mont-Blanc"').values

city_game_coords_df.loc[(city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Garmisch-Partenkirchen"),\
    ['city_ascii', 'lat', 'lng']] = ['Garmisch-Partenkirchen', 47.492, 11.0931]

city_game_coords_df.loc[(city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Moskva"), ['city_ascii', 'lat', 'lng']] = \
    worldcities_df.query('city_ascii == "Moscow"').values

city_game_coords_df.loc[(city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Sankt Moritz"), ['city_ascii', 'lat', 'lng']] = \
    ['St. Moritz', 46.578, 9.8353]

city_game_coords_df.loc[(city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Squaw Valley"), ['city_ascii', 'lat', 'lng']] = \
    ['Palisades Tahoe', 39.208, -120.2132]

city_game_coords_df.loc[(city_game_coords_df['lat'].isna()) & (city_game_coords_df['City'] == "Torino"), ['city_ascii', 'lat', 'lng']] = \
    worldcities_df.query('city_ascii == "Turin"').values

もう一度、緯度経度が入っていないものをチェック。

In [ ]:
city_game_coords_df[city_game_coords_df['lat'].isna()]

都市別開催数の最大値を確認。

In [ ]:
city_game_coords_df['number_of_times'].max()

円の色のバリエーションを用意。

In [ ]:
number_of_times_max = city_game_coords_df['number_of_times'].max()

# 夏季用
color_reds = sns.color_palette("Reds_r", number_of_times_max).as_hex()
# 冬季用
color_blues = sns.color_palette("Blues_r", number_of_times_max).as_hex()

# 色の確認
sns.palplot(color_reds)
sns.palplot(color_blues)

地図を表示。  
地図上のデータからどんなことが読み取れるか？

In [ ]:
map = folium.Map(location=[0,0], zoom_start=2.4)

# 古い円をクリックできるよう、新しいものから順に重ねて描画
for index, row in city_game_coords_df.sort_values('Year', ascending=False).iterrows():
    location = (row['lat'], row['lng'])
    radius = row['number_of_athletes'] * 10
    color = color_reds[row['number_of_times'] - 1] if row['season'] == 'Summer' else color_blues[row['number_of_times'] - 1]

    folium.Circle(location=location,
                  radius=radius,
                  color=color,
                  fill_color=color,
                  weight=1.5,
                  popup=f"{row['City']}\n{row['Year']}\n{row['season']}").add_to(map)
map

### 3. 少しだけ掘り下げた分析をしてみる
細かいところは気にせず、大まかな流れを理解すればOK。

日本の年代別・種類別メダル数がどのように推移しているかを分析。

In [ ]:
# 日本選手に限定
athlete_jpn_df = athlete_events_df.query('NOC == "JPN"')

# 年、メダルでグループ分け
medal_jpn_df = athlete_jpn_df.groupby(['Year', 'Medal']).agg({'ID':'count'}).reset_index()

medal_jpn_df.head()

折れ線グラフで表示。

In [ ]:
# 銅の色はbrownで代用。
sns.lineplot(data=medal_jpn_df, x='Year', y='ID', hue='Medal', hue_order=["Gold", "Silver", "Bronze"],
                palette=["gold", "silver", "brown"])

独自の分析目標を定義して、実際に分析してみよう！

〇〇〇〇〇〇〇〇〇〇を分析。